<a href="https://colab.research.google.com/github/msaeedmt/Fintech/blob/master/Q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
!pip install yfinance

In [79]:
import numpy as np
import yfinance as yf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from math import sqrt
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

In [80]:
def get_historical_klines_with_efficiancy_rate(symbol, start, interval):
  ticker = yf.Ticker(symbol)
  df = ticker.history(start=start,interval=interval)
  df.drop(['Dividends','Stock Splits'], axis=1, inplace=True)
  df['efficiency_rate'] = ((df['Close']-df['Open'])/df['Open']) * 100
  return df

In [81]:
start="2017-01-01"
interval="1d"
btc_df = get_historical_klines_with_efficiancy_rate("BTC-USD", start, interval)

kmeans = KMeans(n_clusters=3)
btc_df["Cluster"] = kmeans.fit_predict(btc_df[['efficiency_rate']])
btc_df["Cluster"] = btc_df["Cluster"].astype("category")
btc_df

,Open,High,Low,Close,Volume,efficiency_rate,Cluster
Date,,,,,,,
2017-01-01,963.658020,1003.080017,958.698975,998.325012,147775008,3.597437,1
2017-01-02,998.617004,1031.390015,996.702026,1021.750000,222184992,2.316503,0
2017-01-03,1021.599976,1044.079956,1021.599976,1043.839966,185168000,2.176976,0
2017-01-04,1044.400024,1159.420044,1044.400024,1154.729980,344945984,10.563956,1
2017-01-05,1156.729980,1191.099976,910.416992,1013.380005,510199008,-12.392691,2
...,...,...,...,...,...,...,...
2022-04-22,40525.863281,40777.757812,39315.417969,39740.320312,28011716745,-1.938374,0
2022-04-23,39738.722656,39935.859375,39352.203125,39486.730469,16138021249,-0.634123,0
2022-04-24,39478.375000,39845.925781,39233.539062,39469.292969,17964398167,-0.023005,0


In [82]:
btc_df['lag_1'] = btc_df['Cluster'].shift(1)
btc_df['lag_2'] = btc_df['Cluster'].shift(2)
btc_df['lag_3'] = btc_df['Cluster'].shift(3)
btc_df['lag_4'] = btc_df['Cluster'].shift(4)
btc_df.dropna(inplace=True)
btc_df

,Open,High,Low,Close,Volume,efficiency_rate,Cluster,lag_1,lag_2,lag_3,lag_4
Date,,,,,,,,,,,
2017-01-05,1156.729980,1191.099976,910.416992,1013.380005,510199008,-12.392691,2,1,0,0,1
2017-01-06,1014.239990,1046.810059,883.943970,902.200989,351876000,-11.046597,2,2,1,0,0
2017-01-07,903.487000,908.585022,823.556030,908.585022,279550016,0.564261,0,2,2,1,0
2017-01-08,908.174988,942.723999,887.249023,911.198975,158715008,0.332974,0,0,2,2,1
2017-01-09,913.244019,913.685974,879.807007,902.828003,141876992,-1.140551,0,0,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-22,40525.863281,40777.757812,39315.417969,39740.320312,28011716745,-1.938374,0,0,0,0,0
2022-04-23,39738.722656,39935.859375,39352.203125,39486.730469,16138021249,-0.634123,0,0,0,0,0
2022-04-24,39478.375000,39845.925781,39233.539062,39469.292969,17964398167,-0.023005,0,0,0,0,0


In [83]:
cols_to_use = []
for i in range(1,5):
  cols_to_use.append('lag_{}'.format(i))

X = btc_df[cols_to_use]
y = btc_df.Cluster

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [90]:
y_train_enc = pd.get_dummies(y_train)
y_test_enc = pd.get_dummies(y_test)

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_test))

In [91]:
model = Sequential()
model.add(Dense(12, input_dim=12, activation='relu'))
model.add(Dense(8 ,activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy','AUC'])

model.fit(OH_cols_train, y_train_enc, epochs=50, batch_size=64)

_, accuracy,auc = model.evaluate(OH_cols_valid, y_test_enc)
print('\nAccuracy: %.2f' % (accuracy*100))
print('AUC: %.2f' % (auc*100))

Epoch 1/50
28/28 [==============================] - 3s 7ms/step - loss: 0.9676 - accuracy: 0.6686 - auc: 0.7615
Epoch 2/50
28/28 [==============================] - 0s 4ms/step - loss: 0.8878 - accuracy: 0.6726 - auc: 0.7639
Epoch 3/50
28/28 [==============================] - 0s 3ms/step - loss: 0.8646 - accuracy: 0.6726 - auc: 0.7739
Epoch 4/50
28/28 [==============================] - 0s 4ms/step - loss: 0.8574 - accuracy: 0.6726 - auc: 0.7772
Epoch 5/50
28/28 [==============================] - 0s 5ms/step - loss: 0.8547 - accuracy: 0.6726 - auc: 0.7781
Epoch 6/50
28/28 [==============================] - 0s 4ms/step - loss: 0.8530 - accuracy: 0.6726 - auc: 0.7792
Epoch 7/50
28/28 [==============================] - 0s 4ms/step - loss: 0.8523 - accuracy: 0.6726 - auc: 0.7797
Epoch 8/50
28/28 [==============================] - 0s 5ms/step - loss: 0.8515 - accuracy: 0.6726 - auc: 0.7798
Epoch 9/50
28/28 [==============================] - 0s 4ms/step - loss: 0.8507 - accuracy: 0.6726 - auc: